In [4]:
from elasticsearch import Elasticsearch, helpers
from elasticsearch.client import IlmClient, ClusterClient, IndicesClient
import requests
import yaml


In [58]:
with open("secrets.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

elastic = Elasticsearch(
    cloud_id=cfg['elastic']['cloudId'],
    api_key=(cfg['elastic']['apiKey'])
)


# Setup

This setup is focused around creating a Time Series Data Stream to write the equilab data to. I'm following the docs [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/set-up-tsds.html), in combination with the python client docs.

## Creating the Index Life Cycle Policy
For the Indexing in Elastic to work correctly there is some setup that needs to be done first. I actually created this policy in the UI and then used the `Show Request` feature in Elastic to get the JSON. That way I had the GUI goodness to help sculpt the JSON, but could still execute it in a notebook.

In [17]:
policy = {
    "phases": {
      "hot": {
        "actions": {
          "rollover": {
            "max_primary_shard_size": "50gb"
          },
          "set_priority": {
            "priority": 100
          },
          "forcemerge": {
            "max_num_segments": 1
          },
          "readonly": {}
        },
        "min_age": "0ms"
      },
      "frozen": {
        "min_age": "30d",
        "actions": {
          "searchable_snapshot": {
            "snapshot_repository": "found-snapshots"
          }
        }
      }
    }
  }


In [18]:
IlmClient.put_lifecycle(elastic, name='equilab-dev', policy=policy)


ObjectApiResponse({'acknowledged': True})

## Mappings Component Template

For TSDS there are special keys that have to be set that cannot be set from the UI, specifically the `time_series_dimension` and `time_series_metric` show below:

In [55]:
mappings = {
    "mappings": {
      "properties": {
        "rideName": {
          "type": "keyword",
          "time_series_dimension": True
        },
        "index": {
            "type": "integer"
        },
        "climb": {
          "type": "float",
          "time_series_metric": "gauge"
        },
        "distance": {
          "type": "float",
          "time_series_metric": "gauge"
        },
        "drop": {
          "type": "float",
          "time_series_metric": "gauge"
        },
        "elevation": {
          "type": "float",
          "time_series_metric": "gauge"
        },
        "speed": {
          "type": "float",
          "time_series_metric": "gauge"
        },
        "timeDelta": {
          "type": "float",
          "time_series_metric": "gauge"
        },
        "@timestamp": {
          "type": "date"
        },
        "location": {
          "type": "geo_point"
        }
      }
    }
  }

mappingsMeta = {
    "description": "Mappings for Equilab data"
  }


In [56]:
ClusterClient.put_component_template(elastic, name='equilab-mappings', template=mappings, meta=mappingsMeta)


ObjectApiResponse({'acknowledged': True})

## Settings Component Template

In [49]:
settings = {
    "settings": {
        "index.lifecycle.name": "equilab-dev",
        "index.look_ahead_time": "3h"
    }
}

settingsMeta = {
    "description": "Settings for Equilab data"
}


In [50]:
ClusterClient.put_component_template(elastic, name='equilab-settings', template=settings, meta=settingsMeta)


ObjectApiResponse({'acknowledged': True})

## Index Template

The Index Template brings it all together, we set the mode to `time_series` and also give a routing path based on the rideName, which will tell Elastic to keep the data for rides on the same node.

In [51]:
templateSettings = {
    "settings": {
        "index.mode": "time_series",
        "index.routing_path": [ "rideName" ]
    }
}
    
templateMeta = {
    "description": "Template for my Equilab data"
}


In [52]:
IndicesClient.put_index_template(
    elastic,
    name='equilab-index-template', 
    index_patterns = ["equilab-rides-*"],
    template = templateSettings,
    composed_of = [ "equilab-mappings", "equilab-settings" ],
    priority = 500,
    meta = templateMeta
)


ObjectApiResponse({'acknowledged': True})